To use the CCI ODP Data Store in a Jupyter Notebook, we need to execute the following lines:

In [1]:
%matplotlib inline
import nest_asyncio
nest_asyncio.apply()

In [2]:
from xcube.core.store.store import find_data_store_extensions
from xcube.core.store.store import get_data_store_params_schema
from xcube.core.store.store import new_data_store
from IPython.display import JSON

Which data accessors are available?

In [3]:
JSON({e.name: e.metadata for e in find_data_store_extensions()})

<IPython.core.display.JSON object>

Usually we need more information to get the actual data store object. Which data store parameters are available?

In [4]:
get_data_store_params_schema('cciodp')

Just provide mandatory parameters to instantiate the store class:

In [5]:
store = new_data_store('cciodp')
store

Which datasets are provided? (the list may contain both gridded and vector datasets):

In [6]:
JSON(list(store.get_data_ids()))

<IPython.core.display.JSON object>

Which in-memory data types are provided?

- `dataset` --> `xarray.Dataset` (Gridded data, multi-resolution pyramid)
- `mldataset` -->  `xcube.core.mlds.MultiLevelDataset`  (Gridded data, multi-resolution pyramid)
- `geodataframe` --> `geopandas.GeoDataFrame` (Vector data) 


In [7]:
store.get_type_ids()

'dataset'

We may ask for a specific type of data ...

In [19]:
store.has_data('esacci.OC.5-days.L3S.CHLOR_A.multi-sensor.multi-platform.MERGED.3-1.geographic')

True

... but in many cases we want to query for certain criteria. How can we do that?

In [20]:
store.get_search_params_schema()

Now search.

In [21]:
iterator = store.search_data(ecv='OZONE', frequency='month')
while True:
    try:
        print(next(iterator))
    except StopIteration:
        break

ClientResponseError: 504, message='Gateway Time-out', url=URL('https://catalogue.ceda.ac.uk/export/xml/4eb4e801424a47f7b77434291921f889.xml')

Which parameters must I pass or are available to open the dataset?

In [22]:
store.get_open_data_params_schema('esacci.OZONE.month.L3.NP.multi-sensor.multi-platform.MERGED.fv0002.r1')

ClientResponseError: 504, message='Gateway Time-out', url=URL('https://catalogue.ceda.ac.uk/export/xml/4eb4e801424a47f7b77434291921f889.xml')

There are 4 required parameters, so we need to provide them to open a dataset:

In [23]:
dataset = store.open_data(
    'esacci.OZONE.month.L3.NP.multi-sensor.multi-platform.MERGED.fv0002.r1', 
    var_names=['surface_pressure'],
    time_range=['2019-05-01', '2019-05-10'])
dataset

ClientResponseError: 504, message='Gateway Time-out', url=URL('https://catalogue.ceda.ac.uk/export/xml/4eb4e801424a47f7b77434291921f889.xml')

In [ ]:
dataset.surface_pressure.isel(time=1).plot.imshow(cmap='Greys', figsize=(16, 16))